# Lecture 9 - metrics used for fairness with Random Forests on COVID data

## Begin with loading packages we need for this example

In [1]:
library(magrittr)
library(caret)
library(splitTools)
library(ROSE)
library(rpart)
library(caret)
library(randomForest)
library(sdgm)

Loading required package: ggplot2

Loading required package: lattice

Loaded ROSE 0.0-4


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: rBayesianOptimization

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine()       masks randomForest::combine()
✖ tidyr::extract()       masks magrittr::extract()
✖ dplyr::filter()        masks stats::filter()
✖ dplyr::lag()           masks stats::lag()
✖ purrr::lift()          masks caret::lift()
✖ randomForest::margin() masks ggplot2::margin()
✖ purrr::set_names()     mask

## Copy the COVID data with complete rows

In [2]:
df <- sdgm::oncovid 
df <- df[complete.cases(df),]
df$case_status <- as.factor(df$case_status)
outcome = "case_status"
head(df)

,case_status,age_group,gender,date_reported,exposure,health_region
,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>
1,0,40-49,Female,285.4583,Close Contact,York Region Public Health Services
2,0,<20,Male,297.4583,Close Contact,York Region Public Health Services
3,0,50-59,Male,274.4583,Not Reported,Peel Public Health
4,0,20-29,Female,260.4583,Close Contact,Halton Region Health Department
5,0,30-39,Female,307.5000,Not Reported,Wellington-Dufferin-Guelph Public Health
6,0,40-49,Female,306.5000,Close Contact,Halton Region Health Department


## Split the data into 60/40 train/test proportions

In [3]:
# split to 60% training and 40% testing
inds <- splitTools::partition(df$case_status, p = c(train = 0.6, test = 0.4))

## Balance the training data

In [4]:
# balance the training data
# df_train <- df[inds$train,]
df_train <- ROSE::ovun.sample(case_status ~., data=df[inds$train,], p=0.5, seed=11, method="both")$data

## Construct the random forest classifier of 100 trees and obtain prediction scores for the test data

### then examine the results…


In [5]:
# build the random forest rf
rf <- randomForest(case_status ~., data= df_train, ntree=100, importance=T)

## Check the classification performance on the test data

In [6]:
# obtain predictions scrores from the random forest for the test data
p1 <- predict(rf, df[inds$test,], 'prob')

# define the desired positive class 
positive_class = "1"

# convert classification scores to decision labels
decisions <- as.factor(ifelse(p1[,positive_class] > 0.5, 1, 0))

# build the confusion matrix
caret::confusionMatrix(decisions, df[inds$test,"case_status"], positive = positive_class)

Confusion Matrix and Statistics

          Reference
Prediction     0     1
         0 30952   150
         1  4045  1245
                                         
               Accuracy : 0.8847         
                 95% CI : (0.8814, 0.888)
    No Information Rate : 0.9617         
    P-Value [Acc > NIR] : 1              
                                         
                  Kappa : 0.3319         
                                         
 Mcnemar's Test P-Value : <2e-16         
                                         
            Sensitivity : 0.89247        
            Specificity : 0.88442        
         Pos Pred Value : 0.23535        
         Neg Pred Value : 0.99518        
             Prevalence : 0.03833        
         Detection Rate : 0.03421        
   Detection Prevalence : 0.14536        
      Balanced Accuracy : 0.88845        
                                         
       'Positive' Class : 1              
                                      

In [7]:
# Inspect what p1 returns -- so you know!
head(p1)

,0,1
2,1,0
4,1,0
5,1,0
6,1,0
9,1,0
10,1,0


## Define a function spd() to calculate Statistical Parity Difference 

In [8]:
# spd() calcualtes Statistical Parity Difference for 
#       group = grp along column = features in data using pos as the positive class label
spd <- function(data, preds, feature, grp, pos){
    
  grp1 <- which(data[c(feature)]==grp)
  grp2 <- which(data[c(feature)]!=grp)
  mean(preds[grp1,pos]) - mean(preds[grp2,pos])
}

## Define a function eod() to calculate Equal Opportunuty Difference

In [9]:
# eod() calcualtes Equal Opportunuty Difference for 
#       group = grp along column = features in data using pos as a class label in outcome column
eod <- function(data, preds, feature, grp, pos){

  # Find rows where the original label is positve    
  x <- which(data[,c(outcome)]== pos)

  # Calcualte spd() difference in predictions for those rows with postive labels
  spd(data[x,],preds[x,],feature,grp,pos)
}

## Define a function aod() to calculate Average Odds Difference

In [10]:
# aod() calcualtes Average Odds Difference for 
#       group = grp along column = features in data using pos as a class label in outcome column
aod <- function(data, preds, feature, grp, pos){

  # find the rows of positives in the desired group
  x <- which(data[,c(outcome)]==positive_class & data[,c(feature)]==grp)

  # find the rows of positives NOT in the desired group
  y <- which(data[,c(outcome)]==positive_class & data[,c(feature)]!=grp)

  # find the rows of negatives in the desired group
  z <- which(data[,c(outcome)]!=positive_class & data[,c(feature)]==grp)

  # find the rows of negatives NOT in the desired group
  w <- which(data[,c(outcome)]!=positive_class & data[,c(feature)]!=grp)

  # return the average difference of their odds
  abs((mean(preds[x,pos]) - mean(preds[y,pos])) + (mean(preds[z,pos]) - mean(preds[w,pos])))/2
}

## Calculate spd(), eod() and aod() for predictions of grp = "80+" in column = "age_group" on the original test data

In [11]:
# define target feature (column)
feature="age_group"

# define target groyp
grp ="80+"

# pp1 <- ifelse(p1 > 0.5, 1, 0)
# SPD() value of age_group = "80+" vs. other age_groups
v11 <- spd(df[inds$test,],p1,feature,grp,positive_class)
v12 <- eod(df[inds$test,],p1,feature,grp,positive_class)
v13 <- aod(df[inds$test,],p1,feature,grp,positive_class)

table(df[inds$test,c(feature,outcome)])

              case_status
age_group         0    1
  <20          4040    0
  20-29        7505    2
  30-39        5717    3
  40-49        5007   11
  50-59        5291   51
  60-69        3337  111
  70-79        1734  243
  80+          2364  974
  Not Reported    2    0

## Induce some bias into age_group = 80+ by removing (excluding) 50% of the those

In [12]:
df_test = df[inds$test,]

# Find which rows in the test data are in age_group = 80+
inds_z = which(df_test[,c(feature)]==grp)

# Find which rows in the test data are NOT in age_group = 80+
inds_notz = which(df_test[,c(feature)]!=grp)

# induce a bias by removing 0.5 of the 80+ age group -- randomly sample 0.5 of those in the group without replacement
inds_nz <- sample(inds_z, size= 0.5 *length(inds_z),replace=F)

# reconstruct the new test data, this is the "biased" test data
new_test_data <- rbind(df_test[inds_nz,], df_test[inds_notz,])

table(new_test_data[,c(feature,outcome)])

              case_status
age_group         0    1
  <20          4040    0
  20-29        7505    2
  30-39        5717    3
  40-49        5007   11
  50-59        5291   51
  60-69        3337  111
  70-79        1734  243
  80+          1189  480
  Not Reported    2    0

## Now, obtain classification scores for the newly modified "biased" test data

In [13]:
# obtain predictions scrores from the random forest for the test data
p2 <- predict(rf, new_test_data, 'prob')

# convert classification scores to decision labels
decisions2 <- as.factor(ifelse(p2[,positive_class] > 0.5, 1, 0))

# build the confusion matrix
caret::confusionMatrix(decisions2, new_test_data[,outcome], positive = positive_class)

Confusion Matrix and Statistics

          Reference
Prediction     0     1
         0 30864   142
         1  2958   759
                                          
               Accuracy : 0.9107          
                 95% CI : (0.9077, 0.9137)
    No Information Rate : 0.9741          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.2995          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.84240         
            Specificity : 0.91254         
         Pos Pred Value : 0.20420         
         Neg Pred Value : 0.99542         
             Prevalence : 0.02595         
         Detection Rate : 0.02186         
   Detection Prevalence : 0.10705         
      Balanced Accuracy : 0.87747         
                                          
       'Positive' Class : 1               
                  

## Compare values of all faireness three metrics

In [14]:

# Fairness metrics values of age_group = "80+" vs. other age_groups
v21 <- spd(new_test_data,p2,feature,grp,positive_class)
v22 <- eod(new_test_data,p2,feature,grp,positive_class)
v23 <- aod(new_test_data,p2,feature,grp,positive_class)

dd <- data.frame(c(v11,v12,v13), c(v21,v22,v23))

colnames(dd) <- c("Original","Modified")
rownames(dd) <- c("SPD","EOD","AOD")

dd

,Original,Modified
,<dbl>,<dbl>
SPD,0.8259157,0.8258139
EOD,0.3150107,0.3106737
AOD,0.5653550,0.5641654
